In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp
from sklearn.linear_model import Ridge

In [2]:
#PreWorked

df = pd.read_csv('salary-train.csv').apply(lambda x: x.astype(str).str.lower())
df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355


In [78]:
vectorizer = TfidfVectorizer(min_df= 5)
X1 = vectorizer.fit_transform(df['FullDescription'])
#X2 = vectorizer.fit_transform(df['LocationNormalized'])
#X3 = vectorizer.fit_transform(df['FullDescription'])

In [79]:
#One-Hot-Coding

df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
#X_test_categ = enc.transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [80]:
# H-stacking

Obj_Feat = sp.sparse.hstack([X1, X_train_categ])

In [81]:
target = df['SalaryNormalized']


In [82]:
rgs = Ridge(alpha= 1 , random_state= 241)

rgs.fit(Obj_Feat, target)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [83]:
Obj_Feat.shape

(60000, 24627)

In [84]:
#Checking

df_test = pd.read_csv('salary-test-mini.csv').apply(lambda x: x.astype(str).str.lower())
df_test['FullDescription'] = df_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
df_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,nan
1,a web developer opportunity has arisen with an...,manchester,permanent,nan


In [85]:
#vectorizer = TfidfVectorizer()
X1_test = vectorizer.transform(df_test['FullDescription'])
#df['LocationNormalized'].fillna('nan', inplace=True)
#df['ContractTime'].fillna('nan', inplace=True)
#enc = DictVectorizer()
X_test_categ = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

Obj_Feat_test = sp.sparse.hstack([X1_test, X_test_categ])

In [86]:
Obj_Feat_test.shape


(2, 24627)

In [88]:
salary_predict = rgs.predict(Obj_Feat_test)

In [89]:
print(round(salary_predict[0],2), round(salary_predict[1],2))

56555.62 37188.32
